# Introduction to machine learning & the supercompressible metamaterial example (3D)

M.A. Bessa (m.a.bessa@tudelft.nl)\
September 30, 2020

**What:** 30min introduction to Machine Learning

**How:** Jointly workout this notebook
* GitHub: https://github.com/bessagroup/f3dasm
    1. You can do this locally in your computer (but you have to have the Python packages installed):
        * clone the repository to your computer: git clone https://github.com/bessagroup/f3dasm
        * load jupyter notebook (it will open in your internet browser): jupyter notebook
        * search for this notebook in your computer (f3dasm/examples/supercompressible/machine_learning) and open it
    2. Or you can use Google's Colab (no installation required, but times out if idle):
        * go to https://colab.research.google.com
        * login
        * File > Open notebook
        * click on Github (no need to login or authorize anything)
        * paste the git link: https://github.com/bessagroup/f3dasm
        * click search and then click on the notebook (f3dasm/examples/supercompressible/machine_learning).


This simple tutorial is based on a script I created for this article: https://imechanica.org/node/23957

It follows from some examples provided by the scikit-learn user guide, which seem to have originated from Mathieu Blondel, Jake Vanderplas, Vincent Dubourg, and Jan Hendrik Metzen.

License: BSD 3 clause

**Let's start by importing basic modules into Python**

## Outline for today

1. Basic steps in machine learning (supervised learning)
2. Our first machine learning model: Gaussian Process regression
3. Is machine learning necessary? Simple example with noisy dataset
4. Replicating the classification plots of the supercompressible metamaterial

## Basic steps in supervised learning

Today we'll cover 4 basic steps in machine learning (supervised learning)

1. Get to know your dataset
2. Pre-process your data (split dataset into **training** and **testing** set)
3. Choose your machine learning method and fit it to the training set.
4. Predict the accuracy of the machine learning model on the testing set.

We will start with a very simple example: regression in 1D

### 1. First look at your dataset

- Here we will create an "artificial" dataset with 50 points from the function $x\sin(x)$ in the domain $x\in[0,10]$.
- In general, we wouldn't know the underlying function (of course!)

In [ ]:
import numpy as np # fundamental scientific computing module
import matplotlib.pyplot as plt # plotting module
import pandas as pd # Pandas dataframe (very common in ML to handle datasets)

# Function to "learn"
def f(x):
    return x * np.sin(x)

n_artificial_data = 50 # number of points in our dataset
x_artificial_data = np.linspace(0, 10, n_artificial_data) # uniformly spaced points
y_artificial_data = f(x_artificial_data) # function values at x_data

#
# Create a dictionary with the names of the input and output variables
# and their respective values:
input_dictionary = {
            'x' : x_artificial_data,
            'y' : y_artificial_data
            }
#
# Then, we create a Pandas data frame:
data = pd.DataFrame(input_dictionary)

So, "data" is our dataset (a Pandas dataframe), just like what F3DASM outputs when it finishes running.

- Given a dataset, before doing anything else make sure you understand the data and its structure (inputs, outputs, names of variables, etc.)

In [ ]:
print(data)

In this dataset "x" are the inputs (features) and "y" are the outputs (targets).

- Save input points (features) in a variable called X_data

- Save output points (targets) in a variable called Y_data 

In [ ]:
x_data = data.loc[:,'x'].values # note that we ask for the
                                # values, not a subset of the
                                # DataFrame

y_data = data.loc[:,'y'].values

print(x_data)
print(y_data)

A short note:

- In most machine learning packages the inputs and outputs should be 2D arrays (even if the variables are 1D vectors, like in this case).

In [ ]:
X_data = np.reshape(x_data, (-1, 1)) # convert vector to 2d array (ML packages use input variables as 2d arrays)

Y_data = np.reshape(y_data, (-1, 1)) # convert vector to 2d array (ML packages use input variables as 2d arrays)

print(X_data)
print(Y_data)

### 2. Pre-process your data

Prior to any learning task, datasets should be pre-processed (scaling, cleaning, etc.).

- Here, we will only do one thing: spliting the data into training and testing data.

A powerful and very popular machine learning package for this is **scikit-learn**.

In [ ]:
from sklearn.model_selection import train_test_split

seed = 1987 # set a random seed so that everyone gets the same result
np.random.seed(seed)

testset_ratio = 0.90 # ratio of test set points from the dataset

# Let's split into 10% training points and the rest for testing:
X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                    Y_data, test_size=testset_ratio,
                                    random_state=seed)

#x_train = X_train.ravel() # just for plotting later
#x_test = X_test.ravel() # just for plotting later

Let's plot the **training** and **testing** points, as well as the underlying function that originated them (usually we don't know the underlying function!)

In [ ]:
fig1, ax1 = plt.subplots() # This opens a new figure

# Plot points and interpolate them:
ax1.plot(X_data, y_data, 'r:', markersize=6, linewidth=2,
         label=u'ground truth: $f(x) = x\,\sin(x)$')

ax1.plot(X_train, Y_train, 'k*', markersize=12,
         label="Training points (data)") # Markers locating training points

ax1.plot(X_test, Y_test, 'ro', markersize=6,
         label="Testing points (data)") # Markers locating training points

ax1.set_xlabel('$x$') # label of the x axis
ax1.set_ylabel('$f(x)$') # label of the y axis
ax1.legend(loc='upper left') # plot legend in the upper left corner

This concludes (a really simple!) dataset pre-processing.

### 3. Choose machine learning model and fit to training set

Suppose that our choice of "machine learning" model is just a simple "polynomial approximation"

- Fitting a polynomial with scikit-learn is trivial!

In [ ]:
# We start by importing the polynomial predictor from scikit-learn
from sklearn.preprocessing import PolynomialFeatures # For Polynomial fit
from sklearn.linear_model import LinearRegression # For Least Squares
from sklearn.pipeline import make_pipeline # to link different objects

degree = 4 # degree of polynomial we want to fit

# Create polynomial model:
poly_model = make_pipeline(PolynomialFeatures(degree),
                           LinearRegression())

# Fit polynomial model to your training data:
poly_model.fit(X_train,Y_train)

Done.

- We have a "machine learning" model that fits our training data.
- Now, let's use our model to predict the testing points (**unseen** by the algorithm)

In [ ]:
Y_test_pred = poly_model.predict(X_test) # prediction of our polynomial
                                         # for unseen points (test data)

print(Y_test_pred)

Let's plot the polynomial prediction on top of fig1.

In [ ]:
# Plot x_data and prediction as a blue line:
ax1.plot(X_test, Y_test_pred, 'bo', linewidth=2,
         label="Prediction at testing points")

ax1.plot(X_data, poly_model.predict(X_data), 'b:', linewidth=2,
         label="Polynomial of degree %d prediction" % degree)

# Replot figure and legend:
ax1.legend(loc='upper left')
fig1

Nice.

Yet, our polynomial (blue) is clearly different of the function we want to "learn", i.e. $x \sin(x)$.

How do we evaluate the quality of our approximation?

### 4. Evaluate model performance

* By evaluating the error of our polynomial model in the points that we didn't use in the fit (testing points).

Two common metrics are $R^2$ and $MSE$

In [ ]:
# Import error metrics:
from sklearn.metrics import mean_squared_error, r2_score

# Compute MSE and R2 for the polynomial model we fitted
mse_value_poly = mean_squared_error(Y_test, Y_test_pred)
r2_value_poly = r2_score(Y_test, Y_test_pred)

print('MSE for polynomial = ', mse_value_poly)
print('R2 score for polynomial = ', r2_value_poly)

As expected, these predictions are not great because:

* We want $MSE$ to be as low as possible

* The closer $R^2$ is to 1.0 the better

## Our first machine learning model

Polynomial models can hardly be considered "machine learning"...

Instead, let's use a machine learning method called Gaussian Processes -- a powerful method for small datasets (<10k points)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ExpSineSquared, ConstantKernel

# Define the kernel function
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # This is the standard RBF kernel
#kernel = 1.0 * RBF(10, (1e-2, 1e2)) # Same kernel as above
                                    #(scikit-learn assumes constant
                                    # variance if you just write RBF
                                    # without the constant kernel or
                                    # without multiplying by 1.0)

# Other examples of kernels:
#kernel = ExpSineSquared(length_scale=3.0, periodicity=3.14,
#                       length_scale_bounds=(0.1, 10.0),
#                       periodicity_bounds=(0.1, 10)) * RBF(3.0, (1e-2, 1e2))
#kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=1.5)
                
gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, n_restarts_optimizer=20) # using a small alpha

# Fit to data using Maximum Likelihood Estimation of the parameters
gp_model.fit(X_train, Y_train)

# Make the prediction on the entire dataset (for plotting)
Y_data_pred, sigma_data_pred = gp_model.predict(X_data, return_std=True) # also output the uncertainty (standard deviation)

SIGMA_data_pred = np.reshape(sigma_data_pred, (-1, 1)) # convert this 1D array into a 2D array for consistency

# Predict for test set (for error metric)
Y_test_pred, sigma_test_pred = gp_model.predict(X_test, return_std=True) # also output the uncertainty (standard deviation)

SIGMA_test_pred = np.reshape(sigma_test_pred, (-1, 1)) # convert this 1D array into a 2D array for consistency

In [ ]:
# Plot the function, the prediction and the 95% confidence interval based on the MSE
fig2, ax2 = plt.subplots() # This opens a new figure

ax2.plot(X_data, Y_data, 'r:', label=u'ground truth: $f(x) = x\,\sin(x)$') # function to learn

ax2.plot(X_train, Y_train, 'ro', markersize=6, label="training points") # noiseless data

ax2.plot(X_test, Y_test, 'kx', markersize=6, label="testing points") # Plot test points

ax2.plot(X_data, Y_data_pred, 'b-', label="GPR prediction")
ax2.fill(np.concatenate([X_data, X_data[::-1]]),
         np.concatenate([Y_data_pred - 1.9600 * SIGMA_data_pred,
                        (Y_data_pred + 1.9600 * SIGMA_data_pred)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')
ax2.set_xlabel('$x$')
ax2.set_ylabel('$f(x)$')
ax2.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=12) # Show in the title the value of the hyperparameters
ax2.legend(loc='upper left')

This looks like a much better prediction than what we obtained with the polynomial fit! In addition, also note that we predict the 95\% confidence interval for our predictions!

- Let's use the regression metrics $R^2$ and $MSE$ to compare the two models (polynomial regression vs GPR)

In [ ]:
# Compute MSE and R2 for the GP model we just fitted:
mse_value_gp = mean_squared_error(Y_test, Y_test_pred)
r2_value_gp = r2_score(Y_test, Y_test_pred)

print('MSE for GP = ', mse_value_gp)
print('R2 score for GP = ', r2_value_gp)


# Printing the previously obtained metrics for the polynomial fit:
print('MSE for polynomial = ', mse_value_poly)
print('R2 score for polynomial = ', r2_value_poly)

The Gaussian Process model is much better!

## Classification plots for the supercompressible metamaterial (3D)

The beautiful thing about modern machine learning packages, is that they are as easy to use for large dimensional datasets as they are for the 1D example that we just covered.

- Let's consider the 3D dataset obtained by F3DASM for the supercompressible metamaterial

In [ ]:
# standard library
import pickle
import itertools

# third-party
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

binary_metrics = ((accuracy_score, 'Accuracy score'),
                  (f1_score, 'F1 score'),
                  (precision_score, 'Precision score'),
                  (recall_score, 'Recall score'))

colors = ['r', 'g', 'b']

Let's import a dataset (previously obtained with F3DASM)

In [ ]:
# get pandas frame
filename = 'DoE_results.pkl'
with open(filename, 'rb') as file:
    data = pickle.load(file)
    
# define number of points to start with
N = 1000

points = data['points'].loc[:(N-1)]
print('variables:', [col for col in points.columns])

# get number of inputs
n_inputs = len(points.columns) - 3

# get X data
X_data = points.iloc[:,range(n_inputs)].values.copy()

# get y data
var_name = 'coilable'
y_data = points.loc[:, var_name].values.copy()

Y_data = np.reshape(y_data, (-1, 1)) # convert vector to 2d array (ML packages use input variables as 2d arrays)

print('X:', X_data)
print('X shape:', np.shape(X_data))
print('Y:', Y_data)
print('Y shape:', np.shape(Y_data))

Since it is a classification problem, let's see how balanced the classes are.

In [ ]:
testset_ratio = 0.20 # ratio of test set points from the dataset

# Let's split into 80% training points and the rest for testing:
X_train, X_test, Y_train, Y_test = train_test_split(X_data,
                                    Y_data, test_size=testset_ratio,
                                    random_state=seed)

Scale data and fit it

In [ ]:
# scale data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# train
clf = SVC(kernel='rbf').fit(X_train_scaled, Y_train.ravel())

Evaluate model with accuracy metric

In [ ]:
def evaluate_model(clf, train_data, test_data, metrics=((accuracy_score, 'Accuracy classification score'),), print_results=True):
    
    # make predictions
    y_train_predicted = clf.predict(train_data[0])
    y_test_predicted = clf.predict(test_data[0])
    
    metric_results = []
    for Imetric in range(np.shape(metrics)[0]):
        metric = metrics[Imetric][0] # select metric used
        metric_results.append([metric(train_data[1], y_train_predicted), metric(test_data[1], y_test_predicted)])
        
        if print_results:
            print("Train", metrics[Imetric][1],"for ML model:", metric_results[-1][0])
            print("Test", metrics[Imetric][1],"for ML model:", metric_results[-1][1])
        
    return metric_results

In [ ]:
# evaluate model
evaluate_model(clf, (X_train_scaled, Y_train), (X_test_scaled, Y_test));

Now, let's plot the result.

In [ ]:
def make_clf_plot(scaler, clf, variables, x3, labels, feature_labels=('Not Coilable','Coilable', 'Coilable (but yields)')):

    column_names = [col for col in points.columns]

    indices =  [column_names.index(name) for name in variables]
    bounds = [data['doe_variables'][name] for name in variables]

    xx = np.meshgrid(np.arange(bounds[0][0], bounds[0][1], 0.005),
                          np.arange(bounds[1][0], bounds[1][1], 0.005))

    n = len(xx[0].ravel())
    xx.append(x3 * np.ones(n))

    XX = np.empty((n, 3))
    for index, xx_ in zip(indices, xx):
        XX[:, index] = xx_.ravel()

    XX_scaled = scaler.transform(XX)

    Y = clf.predict(XX_scaled)
    Y = Y.reshape(xx[0].shape)
    
    n_features = len(feature_labels)
    colors_ = colors[:n_features]
    custom_cm = ListedColormap(colors_)
    
    # plot
    fig, ax = plt.subplots()
    ax.contourf(xx[0], xx[1], Y, cmap=custom_cm, alpha=0.5)
    ax.set_xlabel(labels[0])
    ax.set_ylabel(labels[1])
    ax.set_title('%s = %.3g' % (labels[2], x3))

In [ ]:
plot_variables = ['ratio_pitch', 'ratio_d', 'ratio_top_diameter']
plot_labels = ['$\\frac{P}{D}$', '$\\frac{d}{D_1}$', '$\\frac{D_1 - D_2}{D_1}$']
for x3 in [0., 0.25, 0.5, 0.75]:
    make_clf_plot(scaler, clf, plot_variables, x3, plot_labels)